In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from pyspark.sql import SparkSession
from tqdm import tqdm
import time  # to simulate loading for tqdm

import os
import glob
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col, to_date, count, min, max, lit
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType
from pyspark.sql.functions import col, sum as spark_sum, when

import utils.bronze
import utils.silver

print('Done Importing!')

Done Importing!


In [2]:
# # Create gold table and store to Google Cloud Storage
# gold_label_file_path = "Gold Layer/labels.csv"
# gold_label_gcs_path = f"gs://{bucket_name}/{gold_label_file_path}"

# try:
#     df_labels.to_csv(gold_label_gcs_path, index=False)
#     print("labels.csv Stored to Gold Layer Successfully! ✅")
# except Exception as e:
#     print(f"labels.csv Store Failed: {e}")

# Setup Hadoop
Go to your working directory: CS611_MLE_GROUP_PROJECT <br/>
Then, run this command in your VM terminal:
> wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.20/gcs-connector-hadoop3-2.2.20-shaded.jar -P jars/

<br/><br/>
Copy your google cloud application_default_credentials.json to this working directory, then can run the following commands:

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars", "jars/gcs-connector-hadoop3-2.2.20-shaded.jar") \
        .getOrCreate()

25/06/15 00:58:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# This is required if you are using service account and set true, 
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')
spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', "application_default_credentials.json")
# Following are required if you are using oAuth
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.id', 'YOUR_OAUTH_CLIENT_ID')
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.client.secret', 'OAUTH_SECRET')

# Partition Existing Bronze

In [5]:
# Setup Config
snapshot_date_str = "2015-01-01"

start_date_str = "2015-01-01"
end_date_str = "2017-03-31"

In [6]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
print("Done generate list of dates!")

Done generate list of dates!


In [7]:
dates_str_lst

['2015-01-01',
 '2015-02-01',
 '2015-03-01',
 '2015-04-01',
 '2015-05-01',
 '2015-06-01',
 '2015-07-01',
 '2015-08-01',
 '2015-09-01',
 '2015-10-01',
 '2015-11-01',
 '2015-12-01',
 '2016-01-01',
 '2016-02-01',
 '2016-03-01',
 '2016-04-01',
 '2016-05-01',
 '2016-06-01',
 '2016-07-01',
 '2016-08-01',
 '2016-09-01',
 '2016-10-01',
 '2016-11-01',
 '2016-12-01',
 '2017-01-01',
 '2017-02-01',
 '2017-03-01']

In [8]:
bucket_name = "cs611_mle"

### Bronze Member

In [9]:
### ~~~ Bronze Member ~~~
# create bronze datalake
bronze_member_directory = "datamart/bronze/member"

In [ ]:
# No need to be partitioned
utils.bronze.process_bronze_member(bucket_name, bronze_member_directory, spark)
print('Done Build Bronze Table - Member!')

### Bronze User Log

In [11]:
### ~~~ User Log ~~~
# create bronze datalake
# create multiple csv files 
bronze_userlog_directory = "datamart/bronze/userlog"

In [11]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_userlog(date_str, bucket_name, bronze_userlog_directory, spark)
    
print('Done Build Bronze Table - userlog!')

2015-01-01row count: 2032255


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv


2015-02-01row count: 1812923


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv


2015-03-01row count: 2093862


...

saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_01_01.csv


2017-02-01row count: 2270306


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_02_01.csv


2017-03-01row count: 2587759


saved to: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv
Done Build Bronze Table - userlog!


# Partition Existing Silver

This should have been done in Bronze

### Silver Transaction

In [14]:
### ~~~ Transaction ~~~
# create silver datalake
# create multiple csv files 
bronze_transaction_directory = "datamart/bronze/transaction"

In [15]:

# run bronze backfill
for date_str in dates_str_lst:
    utils.bronze.process_bronze_transaction_partition(date_str, bucket_name, bronze_transaction_directory, spark)
print('Done Partition Silver (Actually Bronze) Table - transaction!')

2015-01-01row count: 97582


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv


2015-02-01row count: 88599


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv


2015-03-01row count: 110650


...

saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_01_01.csv


2017-02-01row count: 116238


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv


2017-03-01row count: 11247


saved to: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv
Done Partition Silver (Actually Bronze) Table - transaction!


# Process Feature Silver Table

### Member ~ Bronze to Silver Table

In [11]:
# create silver datalake
silver_member_directory = "datamart/silver/member/"

In [14]:
utils.silver.process_silver_table_member(bucket_name, bronze_member_directory, silver_member_directory, spark).toPandas().head()

loaded from: gs://cs611_mle/datamart/bronze/member/bronze_members.csv row count: 300000
+------+------+
|gender| count|
+------+------+
|  NULL|171532|
|  male| 65635|
|female| 62833|
+------+------+



saved to: gs://cs611_mle/datamart/silver/member//silver_member.parquet


,msno,city,gender,registered_via,registration_init_time
0,2dwuk0KQUnCnRvFnDGFyIslHLqG8lFHkyOvfRNwc5XI=,5,female,9,2012-03-09
1,Bx13MoR+nVJVVDsatg3fxInFupmO1jSzdkFKMA976t8=,13,female,7,2013-04-22
2,2ouWnOhbWxdFyjEy8qx/8lr15OzZ0PX9tr6xZP4Ihns=,1,na,7,2014-06-29
3,QvZtxnHz7yGh07QiFuq0RN1hL5WrzcbKlQTUiroEEyE=,13,na,9,2011-05-28
4,CUFR22oaKwIwzj4VMey3Mj76B5zawuo2PxsWheNfQ6Q=,5,male,9,2014-11-28


### Transaction ~ Partitioned_Silver to Actual Silver (processed)

In [16]:
actual_silver_transaction_directory = "datamart/silver/transaction/"

In [17]:
for date_str in dates_str_lst:
    utils.silver.process_silver_table_transaction(date_str, bucket_name, bronze_transaction_directory, actual_silver_transaction_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_01_01.csv row count: 97582


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_02_01.csv row count: 88599


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_03_01.csv row count: 110650


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_03_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_04_01.csv row count: 90444


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2015_04_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2015_05_01.csv row count: 74016


...

saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_11_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2016_12_01.csv row count: 138018


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2016_12_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_01_01.csv row count: 160929


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_01_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_02_01.csv row count: 116238


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_02_01.parquet
loaded from: gs://cs611_mle/datamart/bronze/transaction/bronze_transaction_2017_03_01.csv row count: 11247


saved to: gs://cs611_mle/datamart/silver/transaction//silver_transaction_2017_03_01.parquet


### User Log ~ Bronze User Log to Silver User Log

In [9]:
silver_userlog_directory = "datamart/silver/userlog/"


bronze_userlog_directory = "datamart/bronze/userlog"
actual_silver_transaction_directory = "datamart/silver/transaction/"

In [10]:

for date_str in dates_str_lst:
    utils.silver.process_silver_table_userlog(date_str, bucket_name, bronze_userlog_directory, actual_silver_transaction_directory, silver_userlog_directory, spark)

loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_01_01.csv row count: 2032255


Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 1812923


25/06/15 00:58:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2015_01_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2015_02_01.csv row count: 1812923


...

Also loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 2587759


25/06/15 01:05:43 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_02_01.parquet


loaded from: gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_03_01.csv row count: 2587759
File gs://cs611_mle/datamart/bronze/userlog/bronze_userlog_2017_04_01.csv does not exist. Proceeding with single snapshot.


saved to: gs://cs611_mle/datamart/silver/userlog//silver_userlog_2017_03_01.parquet
